# Dissertation Example:

We will use the PCE_calculator module to perform the calculations of section 4.5 in the dissertation.

We start by setting up the input hazard curves and some functions to calculate the mean annual frequency of exceeding limit state thresholds.

In [1]:
import numpy as np
# First step: input the hazard curves values for Seattle (Washington, US) at site class D
# Input Hazard Curves:
## Hazard curve for Sa(T=0.5s) in g
SaT1_0p5 = np.array([3.33e-03,4.99e-03,7.49e-03,1.12e-02,1.69e-02,2.53e-02,3.79e-02,5.69e-02,8.53e-02,1.28e-01,
                     1.92e-01,2.88e-01,4.32e-01,6.48e-01,9.72e-01,1.46e+00,2.19e+00,3.28e+00,4.92e+00,7.38e+00]) # in g
MAF_SaT1_0p5 = np.array([3.26332505e-01, 2.97311836e-01, 2.62163389e-01, 2.23217181e-01, 1.81684122e-01,
                         1.42032902e-01, 1.05926122e-01, 7.51216743e-02, 5.07874237e-02, 3.26517181e-02,
                         1.99433957e-02, 1.14707137e-02, 6.10767896e-03, 2.93842635e-03, 1.24133929e-03,
                         4.42923925e-04, 1.28116361e-04, 2.80302209e-05, 4.07941937e-06, 3.01653082e-07])
## Hazard curve for Sa(T=0.75s) in g
SaT1_0p75 = np.array([2.50e-03, 3.75e-03, 5.62e-03, 8.43e-03, 1.26e-02, 1.90e-02, 2.84e-02, 4.27e-02, 6.40e-02,
                      9.60e-02, 1.44e-01, 2.16e-01, 3.24e-01, 4.86e-01, 7.29e-01, 1.09e+00, 1.64e+00, 2.46e+00,
                      3.69e+00, 5.54e+00]) # in g
MAF_SaT1_0p75 = np.array([3.26007268e-01, 2.96443354e-01, 2.60940080e-01, 2.21371711e-01, 1.80790479e-01,
                          1.40743830e-01, 1.05384221e-01, 7.50928510e-02, 5.13086801e-02, 3.35423384e-02,
                          2.09967194e-02, 1.25368258e-02, 7.05701991e-03, 3.66621784e-03, 1.70930962e-03,
                          6.98245433e-04, 2.35374180e-04, 6.36160157e-05, 1.26119963e-05, 1.56204833e-06])
## Hazard curve for Sa(T=1.0s) in g
SaT1_1 = np.array([2.50e-03, 3.75e-03, 5.62e-03, 8.43e-03, 1.26e-02, 1.90e-02, 2.84e-02, 4.27e-02, 6.40e-02,
                   9.60e-02, 1.44e-01, 2.16e-01, 3.24e-01, 4.86e-01, 7.29e-01, 1.09e+00, 1.64e+00, 2.46e+00,
                   3.69e+00, 5.54e+00]) # in g
MAF_SaT1_1 = np.array([3.05942447e-01, 2.71959035e-01, 2.33464778e-01, 1.92876456e-01, 1.53417414e-01,
                       1.16442445e-01, 8.53182730e-02, 5.97169859e-02, 4.02488727e-02, 2.60456347e-02,
                       1.61888032e-02, 9.62916346e-03, 5.42199893e-03, 2.83358304e-03, 1.33712911e-03,
                       5.55764371e-04, 1.91719768e-04, 5.34495049e-05, 1.10985142e-05, 1.44167810e-06])
## Hazard curve for PGV in cm/s
PGV = np.array([2.370e-01,3.550e-01,5.320e-01,7.980e-01,1.190e+00,1.800e+00,2.690e+00,4.040e+00,6.060e+00,
                9.090e+00,1.360e+01,2.050e+01,3.070e+01,4.600e+01,6.900e+01,1.030e+02,1.550e+02,2.330e+02,
                3.490e+02,5.250e+02]) # in cm/s
MAF_PGV = np.array([2.77855546e-01,2.40999464e-01,2.01858481e-01,1.62792156e-01,1.26685485e-01,9.36685103e-02,
                    6.69436752e-02,4.55074371e-02,2.94349995e-02,1.79787699e-02,1.03234222e-02,5.44864998e-03,
                    2.70331294e-03,1.25159673e-03,5.28095990e-04,1.94727132e-04,5.68743297e-05,1.22216363e-05,
                    1.76541679e-06,1.52154528e-07])

In [2]:
# Second Step: import functions and define the functions needed for the calculations
from PCE_calculator import InputChecker
from PCE_calculator import DataLoader
from PCE_calculator import PCECalculator
from scipy.stats import norm

def interpolate_HCs_SaT(HC1,MAF1,T1,HC2,MAF2,T2,T3):
    """
    Calculate the interpolation between two hazard curves at periods T1 and T2 and returns the hazard curve for period T3

    Args:
    - HC1,HC2 = hazard curve values (Sa(T))
    - MAF1,MAF2 = mean annual frequency of exceedance of each hazard curve
    - T1,T2 : preiod linked to the IM.
    - T3: period of the output hazard curve

    Returns:
    - HC3, MAF3 = hazard curve values (IMs) and mean annual frequency of exceedance of Sa(T3)
    """
    sa_x = np.linspace(min(min(HC1), min(HC2)), max(max(HC1), max(HC2)), 1000)
    MAF1_interp = np.exp(np.interp(np.log(sa_x),np.log(HC1),np.log(MAF1)))
    MAF2_interp = np.exp(np.interp(np.log(sa_x),np.log(HC2),np.log(MAF2)))
    MAF3_interp = np.zeros(1000)
    for i in range(len(MAF1_interp)):
        MAF3_interp[i] = np.exp(np.interp(T3,[T1, T2],[np.log(MAF1_interp[i]), np.log(MAF2_interp[i])]))
    return sa_x, MAF3_interp


def frag_from_coefs(limit, IMs, c1, c2, beta):
    """
    Calculate the fragility function based on the cloud coefficients c1, c2 and beta.
    The fragility function is the probability of exceeding a "limit" given a "IM".
    The formula of the cloud analysis is: ln(displacement) = c1 + c2 * ln(IM) + beta

    Args:
    - limit: limit threshold.
    - c1, c2, beta = cloud coefficients
    - IMs: Vector of intensity measure values (herein, Sd(T1) and PGV).

    Returns:
    - fragility: Vector of the fragility curve given IM and limit.
    """
    # Calculate the argument of the CDF
    argument = (np.log(limit) - (c1 + c2 * np.log(IMs))) / beta

    # Calculate the CDF using scipy.stats.norm.cdf
    fragility = 1 - norm.cdf(argument, 0, 1)

    return fragility

def calculate_MAF(IM,HC,frag,limit):
    """
    Calculate the MAF of exceeding a limit state threshold given a hazard curve and a fragility function.

    Args:
    - IM : vector of IM that the fragility and the hazard curve are defined.
    - HC : vector of hazar curve MAF(IM).
    - frag : fragility curve defined as a function of IM, thus: P(x>threshold | IM)
    - limit: limit threshold.

    Returns:
    - MAF: Mean annual frequency of exceeding the limit
    """
    # Calculate the derivative of the fragility curve:
    derivative_fragility = np.diff(frag) / np.diff(IM)
    derivative_fragility = np.insert(derivative_fragility, 0, 0)
    
    # Calculate the intergral of (HC*dFrag) with trapezium rule
    product = derivative_fragility * HC
    MAF = np.trapz(product,IM)
    
    return MAF

Before starting the calculation we have to make sure that the hazard curves are in the correct unities (meters and meters/sec) and increase its granularity by lineary interpolating them in the log space.

In [3]:
# 3rd step: Hazard curve treatment
## 3.1. Make sure the hazard curve are in the correct unities,
SdT1_0p5 = SaT1_0p5 * 9.81 * (0.5/2/np.pi)**2  # in m
SdT1_0p75 = SaT1_0p75 * 9.81 * (0.75/2/np.pi)**2  # in m
PGV_m = PGV / 100 # in m

## 3.2 Interpolate hazard curve for different periods (if needed)
SaT1_0p9, MAF_SaT1_0p9 = interpolate_HCs_SaT(SaT1_0p75,MAF_SaT1_0p75,0.75,SaT1_1,MAF_SaT1_1,1,0.9)
SdT1_0p9 = SaT1_0p9 * 9.81 * (0.9/2/np.pi)**2  # in m

## 3.3. Make the hazard curves more granular by linear interpolating in the log space
SdT1_0p5_interp = np.linspace(min(SdT1_0p5), max(SdT1_0p5), 1000)
MAF_SdT1_0p5_interp = np.exp(np.interp(np.log(SdT1_0p5_interp),np.log(SdT1_0p5),np.log(MAF_SaT1_0p5)))

SdT1_0p75_interp = np.linspace(min(SdT1_0p75), max(SdT1_0p75), 1000)
MAF_SdT1_0p75_interp = np.exp(np.interp(np.log(SdT1_0p75_interp),np.log(SdT1_0p75),np.log(MAF_SaT1_0p75)))

PGV_interp = np.linspace(min(PGV_m), max(PGV_m), 1000)
MAF_PGV_interp = np.exp(np.interp(np.log(PGV_interp),np.log(PGV_m),np.log(MAF_PGV)))


Now we do the calculations for the first line of Table 4.3 in the dissertation:

In [4]:
# 4th step: calculate MAF of limit states (values of Table 4.3)
# Input for the PCE metamodeL:
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.05 # possible from 0.03 to 0.18
Tb = 3.2 # possible from 3 to 6, in seconds

# Threshold limit for fragility function:
frag_lim_D1 = 0.084 # in meters
frag_lim_u0 = 0.35 # in meters

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 4.80e-10
The MAF(u0>0.3m) is 2.88e-04


The SFP bearing base isolation device that we want to adopt is already good ($T_b$ = 3.2s, $\mu_f$ = 0.05 and capacity = 0.3m). We can try changing the period of the super structure. But we have to have in mind that the period of the base isolation is ideally four times higher than the period of the superstructure ($T_b$ > $4 \times T_1$)

In [5]:
# Change the period repeat the procedure:
## Attention! When we change the period we need to change the hazard curve accordingly:
T1 = 0.75 # s
Tb = 3.2 # s
mratio = 0.55
muf = 0.05

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p75_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p75_interp, MAF_SdT1_0p75_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 1.59e-05
The MAF(u0>0.3m) is 2.90e-04


Now if we change the period of the superstructure to 0.9s the base isolation period should change too to satisfy the requirement that $T_b$ > $4 \times T_1$. The next SFP bearing device in the catalog has $T_b$ = 3.9s.

In [6]:
# Change the period repeat the procedure:
## Attention! When we change the period we need to change the hazard curve accordingly:
T1 = 0.9 # s
Tb = 3.9 # s
mratio = 0.55
muf = 0.05

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p9, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p9, MAF_SaT1_0p9, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 6.97e-05
The MAF(u0>0.3m) is 2.86e-04


We can try a last parameter change. We try changing the friction coefficient to see if it still attains the performance objectives.

In [7]:
# Lets try a final one
T1 = 0.9 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.09 # possible from 0.03 to 0.18
Tb = 3.9 # possible from 3 to 6, in seconds

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p9, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p9, MAF_SaT1_0p9, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 3.25e-04
The MAF(u0>0.3m) is 1.88e-04
